![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_DistilBertForTokenClassification.ipynb)

## Import ONNX DistilBertForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `DistilBertForTokenClassification` is only available since in `Spark NLP 5.1.3` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import DistilBERT models trained/fine-tuned for token classification via `DistilBertForTokenClassification` or `TFDistilBertForTokenClassification`. These models are usually under `Token Classification` category and have `bert` in their labels
- Reference: [TFDistilBertForTokenClassification](https://huggingface.co/transformers/model_doc/distilbert.html#tfdistilbertfortokenclassification)
- Some [example models](https://huggingface.co/models?filter=distilbert&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.29.1`. This doesn't mean it won't work with the future releases
- Albert uses SentencePiece, so we will have to install that as well

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.29.1 optimum tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 15.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.8 MB

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [elastic/distilbert-base-cased-finetuned-conll03-english](https://huggingface.co/elastic/distilbert-base-cased-finetuned-conll03-english) model from HuggingFace as an example
- In addition to `TFDistilBertForTokenClassification` we also need to save the `DistilBertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from optimum.onnxruntime import ORTModelForTokenClassification
import tensorflow as tf

MODEL_NAME = 'elastic/distilbert-base-cased-finetuned-conll03-english'
ONNX_MODEL = f"onnx_models/{ONNX_MODEL}"

ort_model = ORTModelForTokenClassification.from_pretrained(MODEL_NAME, export=True)

# Save the ONNX model
ort_model.save_pretrained(ONNX_MODEL)

Framework not specified. Using pt to export to ONNX.


Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.0.1+cu118
/usr/local/lib/python3.10/dist-packages/transformers/models/distilbert/modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {ONNX_MODEL}

total 255692
-rw-r--r-- 1 root root       960 Oct  3 21:51 config.json
-rw-r--r-- 1 root root 260926482 Oct  3 21:51 model.onnx
-rw-r--r-- 1 root root       125 Oct  3 21:51 special_tokens_map.json
-rw-r--r-- 1 root root       373 Oct  3 21:51 tokenizer_config.json
-rw-r--r-- 1 root root    668923 Oct  3 21:51 tokenizer.json
-rw-r--r-- 1 root root    213450 Oct  3 21:51 vocab.txt


In [ ]:
!mkdir {ONNX_MODEL}/assets

- As you can see, we need to move `vocab.txt` from the tokenizer to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [ ]:
# get label2id dictionary
labels = ort_model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(ONNX_MODEL +'/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

In [ ]:
!mv {ONNX_MODEL}/vocab.txt {ONNX_MODEL}/assets

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [ ]:
!ls -lR {ONNX_MODEL}

onnx_models/elastic/distilbert-base-cased-finetuned-conll03-english:
total 255484
drwxr-xr-x 2 root root      4096 Oct  3 21:51 assets
-rw-r--r-- 1 root root       960 Oct  3 21:51 config.json
-rw-r--r-- 1 root root 260926482 Oct  3 21:51 model.onnx
-rw-r--r-- 1 root root       125 Oct  3 21:51 special_tokens_map.json
-rw-r--r-- 1 root root       373 Oct  3 21:51 tokenizer_config.json
-rw-r--r-- 1 root root    668923 Oct  3 21:51 tokenizer.json

onnx_models/elastic/distilbert-base-cased-finetuned-conll03-english/assets:
total 216
-rw-r--r-- 1 root root     51 Oct  3 21:51 labels.txt
-rw-r--r-- 1 root root 213450 Oct  3 21:51 vocab.txt


## Import and Save DistilBertForTokenClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.1.2
setup Colab for PySpark 3.2.3 and Spark NLP 5.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.3/536.3 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 15.3 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.2.3


- Let's use `loadSavedModel` functon in `DistilBertForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

tokenClassifier = DistilBertForTokenClassification.loadSavedModel(
     ONNX_MODEL,
     spark
 )\
 .setInputCols(["document",'token'])\
 .setOutputCol("ner")\
 .setCaseSensitive(True)\
 .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your DistilBertForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 254860
-rw-r--r-- 1 root root 260966471 Oct  3 21:53 distilbert_classification_onnx
drwxr-xr-x 4 root root      4096 Oct  3 21:53 fields
drwxr-xr-x 2 root root      4096 Oct  3 21:53 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny DitilBertForTokenClassification model 😊

In [ ]:
tokenClassifier_loaded = DistilBertForTokenClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [ ]:
# .getClasses was introduced in spark-nlp==3.4.0
tokenClassifier_loaded.getClasses()

['B-LOC', 'I-ORG', 'I-MISC', 'I-LOC', 'I-PER', 'B-MISC', 'B-ORG', 'O', 'B-PER']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded
])

# couple of simple examples
example = spark.createDataFrame([["My name is Clara and I live in Berkeley, California."], ['My name is Clara and I live in Berkeley, California.']]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "ner.result").show()

+--------------------+--------------------+
|                text|              result|
+--------------------+--------------------+
|My name is Clara ...|[O, O, O, B-PER, ...|
|My name is Clara ...|[O, O, O, B-PER, ...|
+--------------------+--------------------+



That's it! You can now go wild and use hundreds of `DistlBertForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
